# Transfer

In [ ]:
%load_ext jupyter_black
%matplotlib inline

In [1]:
import os
from pathlib import Path
from typing import Iterable

import xarray as xr

import mmmpy


root = Path(os.path.abspath(__name__)).parents[1]
netcdf, binary, grib = sorted(root.glob("data/*.zip"))
with mmmpy.io.unzip(netcdf) as f:
    mrmsn4 = mmmpy.io.read_mrms(f, engine="netcdf4")
with mmmpy.io.unzip(grib) as f:
    mrmsgrib = mmmpy.io.read_mrms(f, engine="cfgrib")
mrmsgrib

ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored
ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored
ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored
ECCODES ERROR   :  Truncating time: non-zero seconds(29) ignored


<xarray.Dataset>
Dimensions:         (heightAboveSea: 2, latitude: 3500, longitude: 7000)
Coordinates:
    time            datetime64[ns] 2015-02-17T23:58:00
    step            timedelta64[ns] 00:00:00
  * heightAboveSea  (heightAboveSea) float64 1.1e+04 1.25e+03
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
    valid_time      datetime64[ns] 2015-02-17T23:58:00
Data variables:
    unknown         (heightAboveSea, latitude, longitude) float32 dask.array<chunksize=(1, 3500, 7000), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161
    history:                 2022-08-19T13:47 GRIB to CDM+CF via cfgrib-0.9.1...

In [42]:
import os
from pathlib import Path
import mmmpy
import xarray as xr
import tarfile
from contextlib import contextmanager
import shutil
import zipfile
import gzip
import uuid
GZIP=".gz" 
ZIP=".zip"
TAR=".tar"
StrPath = str|Path
data = Path(os.path.abspath(__name__)).parents[1] / "data"
file_dest = tuple((file, f"/tmp/mmmpy-{uuid.uuid1()}") for file in data.glob("*"))
file_dest

((PosixPath('/workspaces/mmmpy/data/mosaic3d_tile6_20130531-231500.netcdf.zip'),
  '/tmp/mmmpy-a4cbfa0a-1fd6-11ed-a48f-0242ac110003'),
 (PosixPath('/workspaces/mmmpy/data/mrms_binary_data.zip'),
  '/tmp/mmmpy-a4cbfbea-1fd6-11ed-a48f-0242ac110003'),
 (PosixPath('/workspaces/mmmpy/data/mrms_grib_data.zip'),
  '/tmp/mmmpy-a4cbfcd0-1fd6-11ed-a48f-0242ac110003'),
 (PosixPath('/workspaces/mmmpy/data/index.html'),
  '/tmp/mmmpy-a4cbfdac-1fd6-11ed-a48f-0242ac110003'),
 (PosixPath('/workspaces/mmmpy/data/MRMS_MergedReflectivityQC_00.50_20220818-140041.grib2.gz'),
  '/tmp/mmmpy-a4cbfe7e-1fd6-11ed-a48f-0242ac110003'))

In [17]:
import os
from pathlib import Path
import mmmpy
import xarray as xr
import tarfile
from contextlib import contextmanager
import shutil
import zipfile
import gzip
import uuid
GZIP=".gz" 
ZIP=".zip"
TAR=".tar"
StrPath = str|Path
data = Path(os.path.abspath(__name__)).parents[1] / "data"
def __to_path(*args: StrPath):
    """converts str to Path objects"""
    for arg in args:
        if isinstance(arg, str):
            yield Path(arg)
        else:
            yield arg
@contextmanager
def unzip(file: Path,destination:Path,*, tmpdir: Path = Path("/tmp/mmmpy"), clear_tmp:bool=True):
    file,destination, tmpdir = __to_path(file,destination, tmpdir)

    if not destination.exists():
        destination.mkdir(parents=True)

    if not tmpdir.exists():
        tmpdir.mkdir(parents=True)

    suffixs = tuple(f.casefold() for f in file.suffixes)
    try:
        if ZIP in suffixs:
            with zipfile.ZipFile(file, "r") as zref:
                # dump every thing into the tmpdir
                zref.extractall(destination)

            for file in destination.rglob("*"):
                if GZIP in file.suffixes:
                    newfile = destination / file.name.removesuffix(".gz")
                    with gzip.open(file, "rb") as zref:
                        with newfile.open("wb") as fout:
                            file.unlink()
                            shutil.copyfileobj(zref, fout)

        elif TAR in suffixs:
            with tarfile.open(file, "r") as tar:
                for member in tar.getmembers():
                    if member.isdir():
                        continue
                    # if(n==0):
                    #     n=1
                    #     continue
                    # print(member.linkpath.)
                    fsrc=tar.extractfile(member)
                    newfile = destination / member.name.split("/")[-1].removesuffix(".gz")
                    print(newfile)
                    # print(member.name.split("/")[-1])
                    with newfile.open("wb") as fdst:
                    #         # file.unlink()
                        shutil.copyfileobj(fsrc, fdst)
                    # shutil.copyfileobj(zref, fout)
                    # print(f.)

        yield destination.glob("*")

    finally:
        shutil.rmtree(tmpdir)

def read():
    archive  =  Path(f"/workspaces/mmmpy/archive/")
    # shutil.rmtree(archive)
    for file in data.glob("*.grib2.tar.gz"):
        if "netcdf" in file.name:
            engine = "netcdf4"
        elif "grib" in file.name:
            engine = "cfgrib"
        else:
            engine ="binary"
        with unzip(file, destination=archive, clear_tmp=False) as f:
            # print(tuple(f))
            # print(tuple(f))
            # print(tuple(f))
            x = xr.open_mfdataset(tuple(f),engine=engine)
            print(x)
            # print(tuple(f))
read()    

/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_13.00_20220816-005042.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_11.00_20220816-005042.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_07.00_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_12.00_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_00.50_20220816-005042.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_02.75_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_01.25_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_03.00_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_06.50_20220816-005042.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_17.00_20220816-005042.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_08.00_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_04.00_20220816-005238.grib2
/workspaces/mmmpy/archive/MRMS_MergedRef

Can't create file '/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_13.00_20220816-005042.grib2.923a8.idx'
Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 261, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 328, in message_from_file
    return Message.from_file(file, offset, **kwargs)
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 102, in from_file
    raise EOFError("End of file: %r" % file)
EOFError: End of file: <_io.BufferedReader name='/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_13.00_20220816-005042.grib2'>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 523, in from_indexpath_or_filestream
    self = cls.from_fieldset(filestream, index_keys, computed_ke

EOFError: No valid message found: '/workspaces/mmmpy/archive/MRMS_MergedReflectivityQC_13.00_20220816-005042.grib2'

In [83]:
def read():
    for file, dest in (file, Path(f"archive/")) for file in data.glob("*.grib2.tar.gz"):
        if "netcdf" in file.name:
            engine = "netcdf4"
        elif "grib" in file.name:
            engine = "cfgrib"
        else:
            engine ="binary"
        print(engine)
        with unzip(file, destination= dest, clear_tmp=False) as f:
            # print(tuple(f))
            x = xr.open_mfdataset(f,engine=engine)
            print(x)
            # print(tuple(f))
read()
            

Can't create file '/tmp/mmmpy/mrms-ref3d-grib-2022.grib2.923a8.idx'
Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 523, in from_indexpath_or_filestream
    self = cls.from_fieldset(filestream, index_keys, computed_keys)
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 366, in from_fieldset
    return cls.from_fieldset_and_iteritems(fieldset, iteritems, index_keys, computed_keys)
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 379, in from_fieldset_and_iteritems
    for field_id, raw_field in iteritems:
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 279, in __iter__
    for message in self.itervalues():
  File "/opt/venv/lib/python3.10/site-packages/cfgrib/messages.py", line 255, in itervalues
    with open(self.filestream.path, "rb") as file:
IsADirectoryError: [Errno 21] Is a directory: '/tmp/mmmpy/mrms-ref3d-grib-2022.grib2'
Can't read index file '/t

cfgrib


IsADirectoryError: [Errno 21] Is a directory: '/tmp/mmmpy/mrms-ref3d-grib-2022.grib2'

In [4]:
netcdf, binary, grib = sorted(root.glob("data/*.zip"))
netcdf, binary, grib

(PosixPath('/workspaces/mmmpy/data/mosaic3d_tile6_20130531-231500.netcdf.zip'),
 PosixPath('/workspaces/mmmpy/data/mrms_binary_data.zip'),
 PosixPath('/workspaces/mmmpy/data/mrms_grib_data.zip'))

In [1]:
with mmmpy.io.unzip(netcdf) as f:
    mrms = mmmpy.io.read_mrms(f, engine="netcdf4")
mrms

NameError: name 'mmmpy' is not defined

In [ ]:
mrms= mmmpy.read_mrms(netcdf)
mrms

In [ ]:
@mmmpy.unzip("/tmp/mmmpy/")
def mfread_netcdf(files: Iterable[Path], filetype: str):
    ds = xr.open_mfdataset(
        files,
        engine="netcdf4",
        chunks={},
        backend_kwargs=dict(
            mask_and_scale=True,
            decode_times=True,
            concat_characters=True,
            decode_coords=True,
            use_cftime=None,
            decode_timedelta=None,
            group=None,
            mode="r",
            format="NETCDF4",
            clobber=True,
            diskless=False,
            persist=False,
            lock=None,
            autoclose=False,
        ),
    )
    return ds


mfread_netcdf(netcdf)

In [ ]:
mrms = mmmpy.read_mrms(netcdf)
mrms


In [ ]:


@mmmpy.unzip("/tmp/mmmpy/")
def mfread_cfgrib(files: Iterable[Path], filetype:str):
    ds = xr.open_mfdataset(
        tuple(files)[:2],
        engine="cfgrib",
        concat_dim=["heightAboveSea"],
        combine="nested",
        chunks={},
        # backend_kwargs=None
        backend_kwargs=dict(
            mask_and_scale=True,
            decode_times=True,
            concat_characters=True,
            decode_coords=True,
            # drop_variables=None,
            use_cftime=None,
            decode_timedelta=None,
            lock=None,
            indexpath="{path}.{short_hash}.idx",
            filter_by_keys={},
            read_keys=[],
            encode_cf=("parameter", "time", "geography", "vertical"),
            squeeze=True,
            time_dims=("time", "step"),
        ),
    )
    return ds


mrms = mfread_cfgrib(grib)
mrms